In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\Benjamin Tran\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [3]:
url = 'https://redplanetscience.com/'
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [4]:
# Part 1: NASA Mars News
# Scraping the latest news title
news_title=soup.find_all('div', class_='content_title')[0].text
# Scraping the latest news paragraph
news_paragraph=soup.find_all('div', class_='article_teaser_body')[0].text

In [5]:
news_title

"NASA-JPL Names 'Rolling Stones Rock' on Mars"

In [6]:
news_paragraph

"NASA's Mars InSight mission honored one of the biggest bands of all time at Pasadena concert."

In [7]:
# Part 2: JPL Mars Space Images - Featured Image
url="https://spaceimages-mars.com/"
featured_image_url="https://spaceimages-mars.com/image/featured/mars3.jpg"
browser.visit(url)

In [8]:
html=browser.html
soup=BeautifulSoup(html, 'html.parser')


In [9]:
# Using splinter to locate the correct class and index of the featured image
relative_path = soup.find_all('img')[1]["src"]
featured_image_url = url + relative_path
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [10]:
# Part 3: Mars Facts
# Reading the HTML into pandas
url = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [11]:
# Cleaning up the DataFrame and keeping only what's necessary
mars_facts_df = tables[1]
mars_facts_df.columns = ["Description", "Data"]
mars_facts_df

,Description,Data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
# Converting the cleaned-up DataFrame back to HTML
mars_html= mars_facts_df.to_html()
mars_html
# Removing all the \n in the HTML and printing
mars_html.replace('\n', '')
print(mars_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Data</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
   

In [13]:
# Part 4: Mars Hemispheres
# Mars hemisphere name and image to be scraped
url = 'https://marshemispheres.com/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [14]:
# Scraping hemispheres site and organizing the hemisphere image urls into an empty list
hemispheres=soup.find('div',class_='container')
items=hemispheres.find_all('div',class_='item')
hemisphere_urls=[]

In [15]:
for item in items:
    try:
        # Extract title
        hem=item.find('div',class_='description')
        title=hem.h3.text
        # Extract image url
        hem_url=hem.a['href']
        browser.visit(url+hem_url)
        html=browser.html
        soup=BeautifulSoup(html,'html.parser')
        image_source=url+soup.find('li').a['href']
        if (title and image_source):
        # Print results of title and image link
            print('--------------------')
            print(title)
            print(image_source)
        # Create dictionary for title and url
        dict={'title':title,'image_url':image_source}
        hemisphere_urls.append(dict)
    except Exception as e:
        print(e)


--------------------
Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/full.jpg
--------------------
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
--------------------
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg
--------------------
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg


In [16]:
hemisphere_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [17]:
# Creating a mars dictionary to contain scraped data
Mars_dict = {"News Title":news_title,"News Description":news_paragraph,"Featured Image URL": featured_image_url,"Mars Facts":mars_html,"Hem Images":hemisphere_urls}
Mars_dict

{'News Title': "NASA-JPL Names 'Rolling Stones Rock' on Mars",
 'News Description': "NASA's Mars InSight mission honored one of the biggest bands of all time at Pasadena concert.",
 'Featured Image URL': 'https://spaceimages-mars.com/image/featured/mars1.jpg',
 'Mars Facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Data</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <